In [1]:
import pandas as pd
import numpy as np
import os
import time

In [2]:
reports_dir = 'reports'
def get_file_path(measurement, file_name):
    return os.path.join(reports_dir, measurement, file_name)

In [12]:
def parse_name(name):
    args_list = name.split('-')
    args = {}
    args['Element Type'] = args_list[0]
    args['Interpolation Type'] = args_list[1]
    if args_list[2][0] == '0':
        args['Element Size'] = float('.'+args_list[2][1:])
    else:
        args['Element Size'] = float(args_list[2])
    args['Measurement'] = args_list[3][:-4]
    return args

In [ ]:
nodes_dict = {
    'Node ID': {0: 'A', 1: 'B', 2: 'C', 3: 'D'}, 
    'Orig X': {0: 25.2539, 1: 28.9005, 2: 14.2395, 3: -3.61857}, 
    'Orig Y': {0: -15.0587, 1: 7.19905, 2: 43.8071, 3: -24.8145}, 
    'Orig Z': {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
    }


In [38]:

def read_table_from_file(file_name):

    params = parse_name(file_name)

    if params['Measurement'] == 'sm':
        measurement = 'S, Mises'
    elif params['Measurement'] == 'u':
        measurement = 'U, Magnitude'
    else:
        raise ValueError("Unknown measurement type.")
    
    with open(file_name, 'r') as file:
        lines = file.readlines()

    # Start from the bottom of the file to find where the table begins
    start_line = None
    header_found = False
    for i in range(len(lines)-1, -1, -1):
        if "Part Instance" in lines[i] and "Node ID" in lines[i]:
            start_line = i
            header_found = True
            break

    if not header_found:
        raise ValueError("Table start not found in the file.")
    
    # Extract the header and the table data lines
    header_line = lines[start_line].strip().split()
    table_lines = lines[start_line+1:]

    # Initialize lists to store table data
    part_instances = []
    node_ids = []
    attached_elements = []
    measurements = []

    # Loop over the table lines and parse the data
    for line in table_lines:
        # Assuming the data is space-separated (or adjust if there's a different delimiter)
        columns = line.strip().split()

        # Only process lines with expected number of columns (adjust based on actual table structure)
        if len(columns) >= 4:
            part_instances.append(columns[0])
            node_ids.append(columns[1])
            attached_elements.append(columns[2])
            measurements.append(float(columns[3]))

    node_ids = 

    # Create a DataFrame from the collected data
    data = {
        'Part Instance': part_instances,
        'Node ID': node_ids,
        'Attached elements': attached_elements,
        measurement: measurements,
        'Element Type': np.full(len(part_instances), params['Element Type']),
        'Interpolation Type': np.full(len(part_instances), params['Interpolation Type']),
        'Element Size': np.full(len(part_instances), params['Element Size'])
    }

    df = pd.DataFrame(data)
    
    return df

# Example usage
#file = 'quad-quad-10-sm.rpt'  # Replace with the path to your file
#df = read_table_from_file(file)

# Display the DataFrame
#df

In [40]:
dfs = []
error_counter = 0
for file in os.listdir():
    try:
        df = read_table_from_file(file)
        dfs.append(df)
        #print(df.head())
    except:
        error_counter += 1
        print(f"Failed to read {file}") 

In [35]:
os.chdir('reports')
os.getcwd()

'/Users/martavasconcelos/Documents/Universidade/3º Ano/1º Semestre/MMCom/Projetos/projeto2/reports'

In [106]:
sm = os.listdir()
lin = [file for file in sm if parse_name(file)['Interpolation Type'] == 'lin']
quad = [file for file in sm if parse_name(file)['Interpolation Type'] == 'quad']
lin_tri = [file for file in lin if parse_name(file)['Element Type'] == 'tri']
lin_quad = [file for file in lin if parse_name(file)['Element Type'] == 'quad']
quad_tri = [file for file in quad if parse_name(file)['Element Type'] == 'tri']
quad_quad = [file for file in quad if parse_name(file)['Element Type'] == 'quad']

desired = quad_quad

print(len(desired))
desired

6


['quad-quad-1-sm.rpt',
 'quad-quad-3-sm.rpt',
 'quad-quad-025-sm.rpt',
 'quad-quad-5-sm.rpt',
 'quad-quad-05-sm.rpt',
 'quad-quad-2-sm.rpt']

In [ ]:
file = 'quad-lin'

In [45]:
if '/reports' not in os.getcwd(): os.chdir('reports')
files = os.listdir()
mega_df = pd.DataFrame(columns= ['Node ID', 'Element Type', 'Interpolation Type', 'Element Size', 'U, Magnitude', 'S, Mises'])
for file in files:
    df = read_table_from_file(file)
    mega_df = pd.merge(
        mega_df,
        df, 
        on=['Node ID', 'Element Type', 'Interpolation Type', 'Element Size'], 
        how='outer', 
        suffixes=('', '_drop')
    )

os.chdir('..')

IndexError: list index out of range

In [46]:
from functools import reduce

# Assuming df_list is your list of DataFrames  # Replace with your actual list of DataFrames

# Merge function that applies to all DataFrames in the list
mega_df = reduce(lambda left, right: pd.merge(
    left, 
    right, 
    on=['Part Instance', 'Node ID','Element Type', 'Interpolation Type', 'Element Size'], 
    how='outer', 
    suffixes=('', '_dup')
), dfs)

# Optionally, remove any duplicate columns (if the same information is repeated across DataFrames)
#mega_df = mega_df.loc[:, ~mega_df.columns.duplicated()]

# If you prefer to keep the duplicate columns, use a more specific naming convention in `suffixes`

In [47]:
mega_df

,Part Instance,Node ID,Attached elements,"S, Mises",Element Type,Interpolation Type,Element Size,Attached elements_dup,"U, Magnitude",Attached elements_dup,...,Attached elements_dup,"U, Magnitude_dup",Attached elements_dup,"U, Magnitude_dup",Attached elements_dup,"U, Magnitude_dup",Attached elements_dup,"S, Mises_dup",Attached elements_dup,"S, Mises_dup"
0,OSSO-4,10,10,0.152956,quad,lin,10.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OSSO-4,10,10,0.152956,quad,lin,10.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OSSO-4,10,11,0.152956,quad,lin,10.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OSSO-4,10,11,0.152956,quad,lin,10.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OSSO-4,10,NaN,NaN,tri,lin,10.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,OSSO-4,67,NaN,NaN,tri,quad,0.5,NaN,NaN,NaN,...,7349,0.004219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
726,OSSO-4,67,NaN,NaN,tri,quad,0.5,NaN,NaN,NaN,...,7350,0.004219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
727,OSSO-4,67,NaN,NaN,tri,quad,0.5,NaN,NaN,NaN,...,6015,0.004219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
728,OSSO-4,67,NaN,NaN,tri,quad,0.5,NaN,NaN,NaN,...,7349,0.004219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
mega_df.to_csv('mega_df.csv', index=False)

In [53]:
import pandas as pd
import numpy as np

def read_coordinates_from_file(file_name):

    # Open the file and read all lines
    with open(file_name, 'r') as file:
        lines = file.readlines()

    # Initialize variables for searching the start of the coordinates table
    start_line = None
    header_found = False
    
    # Search for the header line that includes 'Node ID' and 'Orig. Coords'
    for i in range(len(lines)):
        if "Part Instance" in lines[i] and "Node ID" in lines[i] and "Orig. Coords" in lines[i]:
            start_line = i
            header_found = True
            break

    if not header_found:
        raise ValueError("Coordinates table not found in the file.")
    
    # Extract the header and the table data lines
    table_lines = lines[start_line+1:]

    # Initialize lists to store table data
    part_instances = []
    node_ids = []
    orig_coords_x = []
    orig_coords_y = []
    orig_coords_z = []

    # Loop over the table lines and parse the data
    for line in table_lines:
        columns = line.strip().split()

        # Ensure the line has at least 6 columns (Part Instance, Node ID, X, Y, Z)
        # and the third column is not a header or other text (which would raise the ValueError)
        if len(columns) >= 6 and columns[2].replace('.', '', 1).replace('-', '', 1).isdigit():
            part_instances.append(columns[0])
            node_ids.append(columns[1])
            orig_coords_x.append(float(columns[2]))
            orig_coords_y.append(float(columns[3]))
            orig_coords_z.append(float(columns[4]))

    # Create a DataFrame from the collected data
    data = {
        'Node ID': node_ids,
        'Orig X': orig_coords_x,
        'Orig Y': orig_coords_y,
        'Orig Z': orig_coords_z,
    }

    df = pd.DataFrame(data)
    
    return df

# Example usage:
file = 'quad-quad-5-sm.rpt'  # Replace with your file path
df = read_coordinates_from_file(file)

# Display the DataFrame
print(df.to_dict())

{'Node ID': {0: '3', 1: '4', 2: '11', 3: '31'}, 'Orig X': {0: 25.2539, 1: 28.9005, 2: 14.2395, 3: -3.61857}, 'Orig Y': {0: -15.0587, 1: 7.19905, 2: 43.8071, 3: -24.8145}, 'Orig Z': {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}}
